
### 3160100572 蒋柯越

本notebook通过实现Kmeans，DBSCAN两种算法，来完成“海洋工程建模”这门课的第二次聚类方面作业。主要分为几个部分
- 问题总结：回答了课程作业中要求的5个知识点相关问题
- DBSCAN： 通过DBSCAN实现数据集的聚类
- K-means：通过kmeans算法实现聚类。


## 问题总结

- 高维数据样本之间的相异度测算方法；

采用欧氏距离来判断两者之间距离，假设数据有N维：


$d(x,y) = \sqrt{(x_1-y_1)^2+(x_2-y_2)^2+...+ (x_n-y_n)^2}$


- 聚类过程的描述；

DBSCAN和Kmeans的聚类过程，具体在接下来两章中结合代码来进行解释。请直接看代码实现部分。

- 聚类结果的讨论；

这个问题不太清楚。。。聚类结果的评价指标，主要有两种方向。包括外在方法和内在方法，

1. 外在方法：有监督的方法，需要基准数据。用一定的度量评判聚类结果与基准数据的符合程度。也就是说，比如有专家给出了一个已经经过聚类的数据集结果，我们认为这个数据集和他的本分类结果是基准标准，将我们的算法和他们的算法结果进行比较。

这一方面，主要牵涉到，Jaccard系数，FM系数，Rand系数等等。

2. 而内在方法，则是指无监督的方法，这就不需要给出标准的基准方法。主要需要计算两方面的内容：类内聚集程度和类间离散程度。

这一方面的评价，主要有DB指数，Dunn指数，轮廓系数等等。

本文采样DB指数（Davies Bound-in index，DBI）进行比较，具体的计算过程如下：

首先，假设有簇，$\mathrm{C}=\left\{\mathrm{C}_{1}, \mathrm{C}_{2}, \ldots, \mathrm{C}_{\mathrm{k}}\right\}$，我们首先计算簇C内样本间的平均距离
$$\operatorname{avg}(C)=\frac{2}{|C|(|C|-1)} \sum_{1 \leq i \leq j \leq|C|} \operatorname{dist}\left(x_{i}, x_{j}\right)$$

簇C内样本间的最远距离：$\operatorname{diam}(C)=\max _{1 \leq i \leq j \leq|c|} \operatorname{dist}\left(x_{i}, x_{j}\right)$

簇$C_i$与簇$C_j$最近样本间的距离：$d_{\min }(C)=\min _{x_{i} \in C_{i}, x_{j} \in C_{i}} \operatorname{dist}\left(x_{i}, x_{j}\right)$

簇$C_i$与簇$C_j$中心点的距离：$d_{c e n}(C)=\operatorname{dist}\left(\mu_{i}, \mu_{j}\right)$

定义DBI

$$D B I=\frac{1}{k} \sum_{i=1}^{k} \max _{j\not= i}\left(\frac{\operatorname{avg}\left(C_{i}\right)+\operatorname{avg}\left(C_{j}\right)}{d_{\text {cen}}\left(\mu_{i}, \mu_{j}\right)}\right)$$




- 聚类数的讨论；

根据最简单的选择，其实我们的数据样本点较少，可以直接使用$k=\frac{\sqrt{n}}{2}$来进行计算，这样计算得到的结果，大约为3类。在此之外，也可以考虑使用拐点法来进行相关选择，这就涉及到了类内方差的计算。但是因为本文时间有限，因此没有进行尝试。


- 如有可能，请讨论如何评估不同聚类方法？

不同聚类方法的评估方式，已经在“聚类结果的讨论”中进行了阐述。就不做赘述了，在这里，具体分析一下我们用的两种方法的性能。

在最后的结论当中，我讨论了DBSCAN和Kmeans的算法优劣。虽然本次的实验当中，DBSCAN算法的结果比Kmeans差很多（DBSACN的DBI系数大约在2.2左右，Kmeans大约在1.2作用），但整体来说，需要具体根据情况才能判断孰优孰劣。例如，DBSCAN不适合高维数据，Kmeans不适合球形数据，只能进行简单的线性分割。在本实验中，数据的维度很大，为288维，这对于DBSCAN来说非常的不友好，因此，需要具体根据情况判断。

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#  tqdm 为显示训练进度的库
from tqdm import tqdm
from tqdm._tqdm import trange

import numpy as np
import matplotlib.pyplot as plt
import math
import time

数据读取,并简单观察

In [ ]:
data = pd.read_excel('../input/nn-hw1-data/data_hw1.xlsx', sheet_name=0)
data.head()

我们将处理第二个点位的聚类

In [ ]:
data_p2 = data.iloc[:,[0,2]]
data_p2.head()

去掉日期中具体的时间因素，只考虑日期，即288个采样点作为一个样本

In [ ]:
data_p2.iloc[:,0] = data_p2.iloc[:,0].apply(lambda x:x[0:10]) 

In [ ]:
data_p2.head()

In [ ]:
# 为数据添加一列序号
array = np.arange(0,288,1)
array1 = np.arange(0,288,1)
for i in range(int(len(data_p2)/288)-1):
    array1 = np.append(array1,array)
data_p2['index'] = array1

In [ ]:
data_p2.head()

In [ ]:
date = data_p2['Datetime \ Milepost'].unique()

In [ ]:
array = np.zeros((31, 288))

将数据和天数对应，即一天的采样有288个维度的数据

In [ ]:
i=0
for item in date:
    array[i,] = np.array(data_p2[data_p2['Datetime \ Milepost'] == item].iloc[:,1])
    i = i+1
array.shape

In [ ]:
# 标准化
def normalize(x):
    mean = x.mean(axis=0, keepdims=True)
    std = x.std(axis=0, keepdims=True)
    return (x - mean) / std, mean,std

array1, mean, std= normalize(array)

# DBSCAN

在本部分，实现了DBSCAN算法的代码实现（未使用库），并对数据中的第二个点位进行了分类

1. 定义eps作为邻域的半径，MinPts作为判断是否为核心点的临界值。

2. DBSCAN的整体程序流程，是从N个样本点中，随意取一个样本点开始遍历。判断这个样本点是否为核心点（即在其邻域半径eps内，至少有MinPts个点是直接密度可达的），如果是核心点，则找出所有该点直接密度可达的点，成为一个cluster。

3. 继续判断如果找出的样本点，如果找出的样本点也是核心样本点，则循环继续，继续寻找直接密度可达的点，如果找出的点是边界点，则跳出本次循环，继续尚未被分类的点钟提取一个，返回上一步骤（第2步）

4. 当所有点都被遍历了，之后，循环结束。

接下来，具体通过代码来实现该算法。对于每段代码和流程的理解已经卸载了注释里。

In [ ]:
# 定义欧式距离，用于计算两点之间的距离，在后续中和邻域阈值eps进行比较 
UNCLASSIFIED = False
NOISE = 0

def dist(a, b):
    """
    输入：向量a,b
    输出：两个向量的欧式距离
    """
    # print(math.sqrt(np.power(a - b, 2).sum()))
    return math.sqrt(np.power(a - b, 2).sum())

In [ ]:
#  判断输入的两个点是否密度可达
def eps_neighbor(a, b, eps):
    """
    输入：向量a,b,eps
    输出：是否在eps范围内
    """
    return dist(a, b) < eps

In [ ]:
# 记录ID为 pointID的点eps范围内其他密度可达点的id
def region_query(data, pointId, eps):
    """
    输入：数据集, 查询点id, 半径大小
    输出：在eps范围内的点的id
    """
    nPoints = data.shape[1]
    seeds = []
    for i in range(nPoints):
        if eps_neighbor(data[:, pointId], data[:, i], eps):
            seeds.append(i)
    return seeds

In [ ]:
# 首先判断pointID是否是核心点，如果是核心点，首先将新的核心点划分进前一点的cluster之内， 
def expand_cluster(data, clusterResult, pointId, clusterId, eps, minPts):
    """
    输入：数据集, 分类结果, 待分类点id, 簇id, 半径大小, 最小点个数
    输出：能否成功分类
    """
    seeds = region_query(data, pointId, eps)
    # 不满足minPts条件设置为噪声点，分类标签NOISE = 0
    if len(seeds) < minPts: 
        clusterResult[pointId] = NOISE
        return False
    #  如果 pointID是核心点，进行后续操作
    else:
        clusterResult[pointId] = clusterId # 划分到该簇
        for seedId in seeds:
            clusterResult[seedId] = clusterId
 
        # 在邻域内的其他点，
        while len(seeds) > 0: # 持续判断
            currentPoint = seeds[0]
            queryResults = region_query(data, currentPoint, eps)
            # 如果当前的点是核心
            if len(queryResults) >= minPts:
                for i in range(len(queryResults)):
                    resultPoint = queryResults[i]
                    if clusterResult[resultPoint] == UNCLASSIFIED:
                        seeds.append(resultPoint)
                        clusterResult[resultPoint] = clusterId
                    elif clusterResult[resultPoint] == NOISE:
                        clusterResult[resultPoint] = clusterId
            # 去除当前已经遍历过的点
            seeds = seeds[1:]
        return True

以下即为DBSCAN的主体函数，具体算法细节在之前的几个函数中已经 体现出了 ，这个DBSCAN函数 ，其实做的就是遍历所有节点，当所有节点的标签都被贴上之后 ，即可结束循环。

In [ ]:
def DBSCAN(data, eps, minPts):
    """
    输入：数据集, 半径大小, 最小点个数
    输出：分类簇id
    """
    clusterId = 1
    nPoints = data.shape[1]
    print("points: {}".format(nPoints))
    clusterResult = [UNCLASSIFIED] * nPoints
    for pointId in range(nPoints):
        point = data[:, pointId]
        # 判断是否已经完成分类
        if clusterResult[pointId] == UNCLASSIFIED:
            if expand_cluster(data, clusterResult, pointId, clusterId, eps, minPts):
                clusterId = clusterId + 1
    return clusterResult, clusterId - 1
 

In [ ]:
# 使用 DBSCAN进行分类
dataset = array1.transpose()
clusters, clusterNum = DBSCAN(dataset, 15, 8)
print("clusterResult:{} cluster Numbers = {}".format(clusters,clusterNum))

In [ ]:
from sklearn.metrics import davies_bouldin_score

davies_bouldin_score(array1, clusters)

### DBSCAN结论

其实DBSCAN的效果并不是很好，查阅相关资料之后，大家都提到了DBSCAN对于高维数据的处理并不是特别好。主要原因，因为高维数据中的“距离”概念十分抽象且模糊，难以得到很完整的优化和妥善处置。

因此，接下来换用K-means，来试试能不能得到一些更好的效果。

# k-means

在本部分，实现了kmeans算法的代码实现（未使用库），对数据集中的同样数据也完成了分类

K-means的算法结构其实也比较简单。

1. 随机取k个点作为聚类中心
2. 计算其他点到聚类中心距离，待标记点的和哪个聚类中心的距离最近，则归类到该聚类中心点所在的簇(cluster)去。从而进行相关分类工作。
3. 计算同一簇（cluster）内的平均值，作为新的聚类中心。
4. 重复 2，3，如果最后聚类中心不变化，则算法结束。

接下来，代码中就是这样做的，dist用于计算向量间距离，和DBSCAN一样，randCent则用于初始化 聚类中心

In [ ]:
from numpy import *
# 计算欧几里得距离
def dist(a, b):
    return math.sqrt(np.power(a - b, 2).sum())

# 初始化 聚类中心
def randCent(dataSet, k):
    n = np.shape(dataSet)[1]
    centroids = mat(zeros((k,n)))   # 每个聚类中心有n个坐标值，总共要k个中心
    for j in range(n):
        minJ = min(dataSet[:,j])
        maxJ = max(dataSet[:,j])
        rangeJ = float(maxJ - minJ)
        centroids[:,j] = minJ + rangeJ * random.rand(k, 1)
    return centroids

In [ ]:
# k-means 聚类算法
def kMeans(dataSet, k):
    m = np.shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))    # 用于存放该样本属于哪类及距离
    # clusterAssment第一列存放其所属的中心点，第二列是该数据点到中心点的距离
    centroids = randCent(dataSet, k)
    clusterChanged = True   # 用来判断前后中心是否一致，即聚类是否已经收敛
    while clusterChanged:
        clusterChanged = False;
        for i in range(m):  # 把每一个数据点划分到离它最近的中心点
            minDist = inf; minIndex = -1;
            for j in range(k):
                # 计算到聚类中心距离
                distJI = dist(centroids[j,:], dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j  # 如果第i个数据点到第j个中心点最近，则将i聚类为j
            if clusterAssment[i,0] != minIndex: 
                clusterChanged = True  # 如果分配发生变化，则需要继续迭代
            clusterAssment[i,:] = minIndex,minDist**2   # 并将第i个数据点的分配情况存入字典
        for cent in range(k):   # 重新计算中心点
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A == cent)[0]]   # 去第一列等于cent的所有列
            centroids[cent,:] = mean(ptsInClust, axis = 0)  # 算出这些数据的中心点
    return centroids, clusterAssment


In [ ]:
# 用测试数据及测试kmeans算法
myCentroids,clustAssing = kMeans(dataset.transpose(),3)
print(clustAssing[:,0].transpose())

In [ ]:
# 计算DBI
davies_bouldin_score(array1, clustAssing[:,0])

可以发现，其实这个效果已经远好于DBSCAN了。这说明，这里的数据可能还是比较适合使用KMeans，而不是DBSCAN（如数据并不需要球形、噪声点也比较少这种情况）

同样是用sklearn的KMeans进行一簇聚类，看看效果。

In [ ]:
from sklearn.cluster import KMeans
y_pred_km = KMeans(n_clusters=3, random_state=9).fit_predict(array1)
y_pred_km

In [ ]:
davies_bouldin_score(array1, y_pred_km)

可以注意到，两次聚类最后得到的效果整体来说差不多，虽然有着及其微小的区别。这些区别主要是来源于初始化的区别而产生的。因为，在Kmeans开始之前，我们需要进行一次聚类中心初始化的过程。而这也是Kmeans一个较大的弊端。但是，整体来说，聚类结果还是类似（忽略掉标号的区别，因为聚类算法其实标签本身没有任何意义。）

于是 ，综上所示，在这次数据聚类的实验当中，DBSCAN和Kmeans两者相比较，其实性能孰优孰劣难以很完整地评价。DBSCAN 其实很难适应高维度数据，Kmeans只能进行线性的聚类。虽然最后，KMeans的结果远远好于DBSCAN，但我个人还是觉得，如果不经过PCA等等数据集降维就直接下结论，可能并不是很合适。但 作业时间有限，也就不深究了。